# N-Gram Language Modeling

In **language modeling** we want to model the probability of variable length sequences, $$\large p(w_1,\ldots,w_T)=\prod_{t=1}^T p(w_t|w_{<t}).$$

An **n-gram language model** assumes that each word $w_t$ only depends on the preceding $n-1$ words, $$\large p(w_1,\ldots,w_T)=\prod_{t=1}^T p(w_t|w_{t-n+1},\ldots,w_{t-1}).$$

 

#### Example
For instance, when modeling the sentence $$\texttt{the cat sat on the mat .}$$ a 3-gram language model assumes that $$p(\texttt{mat}|\texttt{the cat sat on the}) \approx p(\texttt{mat}|\texttt{on the}).$$

The sub-sequence $(\texttt{on the mat})$ is a *3-gram* or *trigram*.

### Count-based Estimation

Given some dataset $D$ of sequences, we can estimate an n-gram model through counting, derived as follows:

\begin{align}
p(w_t|w_{t-n+1},\ldots,w_{t-1}) &= \frac{p(w_{t-n+1},\ldots,w_t)}{p(w_{t-n+1},\ldots,w_{t-1})} & \text{definition of conditional probability}\\
                       &= \frac{p(w_{t-n+1},\ldots,w_t)}{\sum_{w_{t'}}p(w_{t-n+1},\ldots,w_{t-1},w_{t'})}\\
                       &\approx \frac{\frac{1}{N}\text{count}(w_{t-n+1},\ldots,w_t)}{\frac{1}{N}\sum_{w_{t'}}\text{count}(w_{t-n+1},\ldots,w_{t-1},w_{t'})}\\
                       &= \frac{\text{count}(w_{t-n+1},\ldots,w_t)}{\sum_{w_{t'}}\text{count}(w_{t-n+1},\ldots,w_{t-1},w_{t'})}\\
                       &= \frac{\text{count}(w_{t-n+1},\ldots,w_t)}{\text{count}(w_{t-n+1},\ldots,w_{t-1})},
\end{align}

where $N$ is the number of $n$-grams in the dataset.

In Python, we can collect these counts into a dictionary mapping a prefix to a dictionary of counts:

        count[(w_n+1,...,w_t-1)] = {wt1: count of (w_n+1,...,w_t1),
                                    wt2: count of (w_n+1,...,w_t2),
                                    ...
                                   }
                                   
and for the denominator, maintain a dictionary of totals:

        total[(w_n+1,...,w_t-1)] = count of w_n+1,...,w_t-1

Now that we've discussed some preliminaries, let's get started importing some basic packages:

In [1]:
%pylab inline
from collections import defaultdict
import json
import numpy as np
import os
import pandas as pd
from tqdm.auto import tqdm
pd.set_option('display.max_colwidth', -1)

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


### Simplified Language

To get intuition, lets start by modeling a simple language called `ABC`. A word in this language is one of three tokens, $$w\in \{\texttt{A, B, C}\},$$
and we'll denote a sentence as $\textbf{w}=(w_1,\ldots,w_{|\textbf{w}|})$.


Suppose we are given the following dataset, and want to estimate a **bigram model**: $$p(\textbf{w})\approx\prod_{t=1}^{|\textbf{w}|}p(w_t|w_{t-1})\quad\quad(*)$$

In [4]:
data_raw = [['A', 'A', 'B', 'B'],
            ['A', 'A', 'B'],
            ['A', 'A', 'B', 'C'],
            ['A', 'A', 'A'],
            ['A', 'A', 'A', 'A']]

Since our model is a probability distribution, the total probability of all possible strings in the language must sum to 1, i.e.: $$\sum_{\textbf{w}}p(\textbf{w})=1.$$

In order to satisfy this criterion it turns out that we need an additional **beginning token**, `<bos>`, and **end token**, `<eos>`:

In [5]:
data = [['<bos>'] + d + ['<eos>'] for d in data_raw]
data

[['<bos>', 'A', 'A', 'B', 'B', '<eos>'],
 ['<bos>', 'A', 'A', 'B', '<eos>'],
 ['<bos>', 'A', 'A', 'B', 'C', '<eos>'],
 ['<bos>', 'A', 'A', 'A', '<eos>'],
 ['<bos>', 'A', 'A', 'A', 'A', '<eos>']]

Now let's estimate a bigram model:

\begin{align}
p(w_t|w_{t-1}) &= \frac{\text{count}(w_{t-1}w_{t})}{\sum_{w_{t'}}\text{count}(w_{t-1}w_{t'})}\\
               &= \texttt{count[prefix][wt] / totals[prefix]}
\end{align} 

where $\texttt{prefix}$ is $w_{t-1}$ in this case.

In [6]:
count = defaultdict(lambda: defaultdict(float))
total = defaultdict(float)

n = 2
for sequence in data:
    for i in range(len(sequence)-n+1):         # for each ngram
        ngram = tuple(sequence[i:i+n])
        prefix, word = ngram[:-1], ngram[-1]
        count[prefix][word] += 1               # count(w_{t-n+1}...w_t)
        total[prefix] += 1                     # count(w_{t-n+1}...w_{t-1})

Let's see if the counts and totals make sense:

- How many times did (A, B) occur? What about (B, B)?
- How many times did (A) occur? What about (C)?

In [7]:
print("Counts:")
dict(count)

Counts:


{('<bos>',): defaultdict(float, {'A': 5.0}),
 ('A',): defaultdict(float, {'<eos>': 2.0, 'A': 8.0, 'B': 3.0}),
 ('B',): defaultdict(float, {'<eos>': 2.0, 'B': 1.0, 'C': 1.0}),
 ('C',): defaultdict(float, {'<eos>': 1.0})}

In [8]:
print("\nTotals:")
dict(total)


Totals:


{('<bos>',): 5.0, ('A',): 13.0, ('B',): 4.0, ('C',): 1.0}

#### Conditional probability queries

We can now query a conditional probability:

\begin{align}
\texttt{p(word|prefix)} =&\ \texttt{count[prefix][word] / totals[prefix]}
\end{align}

In [9]:
queries = [('<bos>', 'A'),
           ('B', 'C')]

for query in queries:
    prefix, word = query[:-1], query[-1]
    p = count[prefix][word] / total[prefix]  # We'll discuss the case when `total[prefix] = 0` below.
    print("p( %s | %s) = \t%.5f" % (word, ', '.join(prefix), p))

p( A | <bos>) = 	1.00000
p( C | B) = 	0.25000


**Exercise**: Look at the training set and convince yourself that these conditional probabilities are correct according to the count-based estimation procedure.

#### Sequence Probability

We can compute the probability of a sequence using the conditional probabilities along with the chain rule of probability:

\begin{align}
p(w_1,\ldots,w_T)&\approx\prod_{t=1}^T p(w_t|w_{t-1})
\end{align}

(Here $w_0$ is `<bos>` and $w_T$ is `<eos>`)

In [10]:
sequence = ['<bos>', 'A', 'A', 'B', '<eos>']

def sequence_p(sequence, log=False):
    total_p = 1

    for i in range(len(sequence)-n+1):
        ngram = tuple(sequence[i:i+n])
        prefix = ngram[:-1]
        word = ngram[-1]
        p = count[prefix][word] / max(total[prefix], 1)
        if log:
            print("p(%s | %s) =\t%.3f" % (word, ', '.join(prefix), p))

        total_p *= p
    return total_p
    

print("\nProduct: p(%s) = %.3f" % (''.join(sequence[1:-1]), sequence_p(sequence, log=True)))

p(A | <bos>) =	1.000
p(A | A) =	0.615
p(B | A) =	0.231
p(<eos> | B) =	0.500

Product: p(AAB) = 0.071


### Real Example: Dialogue Utterances

Now lets use the same ideas on a more realistic text corpus.

We will use utterances from a dialogue dataset called **Persona-Chat**. This dataset is relatively small and centers on a single domain, but it is simple and interpretable for our purposes here. You can download it as follows:

In [11]:
if not os.path.exists('personachat_all_sentences_train.jsonl'):
    !wget "https://nyu.box.com/shared/static/q4nvswb0szelivhgyx87vd1056ttqfyi.jsonl" -O 'personachat_all_sentences_train.jsonl'
if not os.path.exists('personachat_all_sentences_valid.jsonl'):
    !wget "https://nyu.box.com/shared/static/8krcizo8sms1m0ppy7uiwfcx4a3l5nsq.jsonl" -O 'personachat_all_sentences_valid.jsonl'

--2021-05-16 21:45:51--  https://nyu.box.com/shared/static/q4nvswb0szelivhgyx87vd1056ttqfyi.jsonl
Resolving nyu.box.com (nyu.box.com)... 107.152.26.197
Connecting to nyu.box.com (nyu.box.com)|107.152.26.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/q4nvswb0szelivhgyx87vd1056ttqfyi.jsonl [following]
--2021-05-16 21:45:51--  https://nyu.box.com/public/static/q4nvswb0szelivhgyx87vd1056ttqfyi.jsonl
Reusing existing connection to nyu.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://nyu.app.box.com/public/static/q4nvswb0szelivhgyx87vd1056ttqfyi.jsonl [following]
--2021-05-16 21:45:52--  https://nyu.app.box.com/public/static/q4nvswb0szelivhgyx87vd1056ttqfyi.jsonl
Resolving nyu.app.box.com (nyu.app.box.com)... 107.152.26.201
Connecting to nyu.app.box.com (nyu.app.box.com)|107.152.26.201|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://public.boxcloud

#### Loading the dataset

In [12]:
train = []
valid = []

for ds, name in [(train, 'train'), (valid, 'valid')]:
    with open('personachat_all_sentences_%s.jsonl' % name, 'r') as f:
        for line in f:
            ds.append(json.loads(line)['tokens'])
        
vocab = list(set([t for ts in train for t in ts]))      
print("Number of train examples: %d" % (len(train)))
print("Number of valid examples: %d" % (len(valid)))
print("Vocab size: %d" % (len(vocab)))

print("\nExamples:")
train[:3]

Number of train examples: 133176
Number of valid examples: 16181
Vocab size: 19153

Examples:


[['i', 'am', 'doing', 'great', 'except', 'for', 'the', 'allergies', '.'],
 ['i', 'am', 'a', 'woman', 'what', 'about', 'you', '.'],
 ['i', 'thought', 'you', 'were', 'a', 'college', 'kid', '.']]

#### Convert tokenized data to input

In [13]:
os.makedirs('data/tokenized', exist_ok=True)
with open('data/tokenized/pchat_train', 'w') as f:
    for line in tqdm(train):
        f.write(' '.join(line))
        f.write('\n')

### KenLM

We'll use an off-the-shelf ngram modeling package called `KenLM`. KenLM estimates n-gram language models using **modified Kneser-Ney smoothing**, and has a fast and memory-efficient implementation. 
- While we won't go into details here, **smoothing** is a technique used to account for ngrams that do not occur in the training corpus. 
- Normally, these ngrams would receive zero-probability mass. Smoothing ensures every ngram receives some probability.



Please see page 48 of the [lecture note](https://github.com/nyu-dl/NLP_DL_Lecture_Note/blob/master/lecture_note.pdf) for an overview of Kneser-Ney smoothing, and [[Chen & Goodman 1998]](https://dash.harvard.edu/bitstream/handle/1/25104739/tr-10-98.pdf?sequence=1) for further details.

Let's install KenLM below:

In [14]:
!wget -O - https://kheafield.com/code/kenlm.tar.gz |tar xz
!mkdir /content/kenlm/build; cd /content/kenlm/build; cmake ..; make -j 4
!cd /content/kenlm; python setup.py install
KENLM_DIR='/content/kenlm'

!pip install pypi-kenlm

--2021-05-16 21:45:58--  https://kheafield.com/code/kenlm.tar.gz
Resolving kheafield.com (kheafield.com)... 35.196.63.85
Connecting to kheafield.com (kheafield.com)|35.196.63.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 491090 (480K) [application/x-gzip]
Saving to: ‘STDOUT’

-                   100%[===================>] 479.58K  --.-KB/s    in 0.03s   

2021-05-16 21:45:58 (16.6 MB/s) - written to stdout [491090/491090]

-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compil

#### Build kenlm n-gram models

This uses the `kenlm` commands `lmplz` to estimate the language model, then `build_binary` to convert it to an efficient format. We load the resulting model using the `kenlm` python wrapper.

We do this for n-gram models of order `2,3,4`:

In [15]:
import kenlm

data_prefix = 'pchat'
dataset = 'pchat_train'
models_dir = 'models'
os.makedirs(models_dir, exist_ok=True)

models = {}
for n in [2,3,4]:
    model_temp = '%s/%s_%d.arpa' % (models_dir, data_prefix, n)
    model_name = '%s/%s_%d.klm' % (models_dir, data_prefix, n)
    ! cat ./data/tokenized/$dataset | $KENLM_DIR/build/bin/lmplz -o $n > $model_temp
    ! $KENLM_DIR/build/bin/build_binary $model_temp $model_name
    models[n] = kenlm.LanguageModel(model_name)

=== 1/5 Counting and sorting n-grams ===
File stdin isn't normal.  Using slower read() instead of mmap().  No progress bar.
tcmalloc: large alloc 2965790720 bytes == 0x5638dfaf8000 @  0x7f58a6da71e7 0x5638dea567a2 0x5638de9f151e 0x5638de9d02eb 0x5638de9bc066 0x7f58a4f40bf7 0x5638de9bdbaa
tcmalloc: large alloc 7908777984 bytes == 0x56399075e000 @  0x7f58a6da71e7 0x5638dea567a2 0x5638dea457ca 0x5638dea46208 0x5638de9d0308 0x5638de9bc066 0x7f58a4f40bf7 0x5638de9bdbaa
Unigram tokens 1602042 types 19156
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:229872 2:10899497984
tcmalloc: large alloc 10899505152 bytes == 0x563b68674000 @  0x7f58a6da71e7 0x5638dea567a2 0x5638dea457ca 0x5638dea46208 0x5638de9d08d7 0x5638de9bc066 0x7f58a4f40bf7 0x5638de9bdbaa
Statistics:
1 19156 D1=0.588857 D2=1.0348 D3+=1.3388
2 231267 D1=0.708637 D2=1.06132 D3+=1.37629
Memory estimate for binary LM:
type      kB
probing 4551 assuming -p 1.5
probing 4626 assuming -r models -p 1.5
trie    1747 witho

## Evaluation

### Perplexity

Intuitively, a good model should assign high probabilities to sequences from the 'true' distribution that it is modeling.

A common way of quantifying this is with **perplexity**, a metric inversely-proportional to the probability that the model assigns to a set of sequences, e.g. a 'test set':

\begin{align}
\large \text{ppl}(p, D) &\large\ = 2^{-\frac{1}{N_{total}}\log_2 p(D)}
\end{align}


where $D=\{(w_1,\ldots,w_{N_i})_i\}_{i=1}^M$ is a dataset of $M$ sequences with total length $N_{\text{total}}=\sum_{i}N_i$.

Perplexity is defined on $[1,\infty)$, with 1 being a perfect model (assigning probability 1 to $D$), and a 'worse' model as perplexity increases.

Intuitively, _perplexity measures the average rank of the true next-token, when tokens are ordered by the model's conditional probabilities_. Another intuitve interpretation of perplexity is that it measures the average number of times you would need to sample from the language model in order to receive the true next token.


<!-- Section 1.3 of [[Chen & Goodman 1998]](https://dash.harvard.edu/bitstream/handle/1/25104739/tr-10-98.pdf?sequence=1) has a concise summary of perplexity and its motivation. !-->

#### Evaluate Perplexity

`kenlm` outputs log probabilities in **log base 10**. For the standard definition of perplexity we need **log base 2**. See the code below:

In [16]:
def perplexity_kenlm(model, sequences):
    n_total = 0
    logp_total = 0
    for sequence in sequences:
        text = ' '.join(sequence)
        logp_total += model.score(text)
        n_total += len(sequence) + 1  # add 1 for <eos>
        
    # Convert log10 to log2
    log2p_total = logp_total / np.log10(2)
    ppl = 2 ** (- (1.0 / n_total) * log2p_total)
    return ppl

In [17]:
print("=== Train ===")
df = pd.DataFrame([(n, perplexity_kenlm(models[n], train)) for n in [2, 3, 4]], columns=['n', 'ppl'])
df.style.hide_index()

=== Train ===


n,ppl
2,39.924842
3,15.462037
4,8.919611


In [18]:
print("=== Valid ===")
df = pd.DataFrame([(n, perplexity_kenlm(models[n], valid)) for n in [2, 3, 4]], columns=['n', 'ppl'])
df.style.hide_index()

=== Valid ===


n,ppl
2,59.187973
3,44.977117
4,43.218831


#### Sequence probabilities:
\begin{align}
p(w_1,\ldots,w_T)&\approx\prod_{t=1}^T p(w_t|w_{t-2},w_{t-1})\\
\log p(w_1,\ldots,w_T) &\approx \sum_{t=1}^T \log p(w_t|w_{t-2},w_{t-1}).
\end{align}

where we use log probabilities in practice to avoid a product of many small numbers.

In [19]:
sentences = [
    'i like my pet dog .',
    'i like my pet zebra .',
    'i like my pet lion .',
    'i live in the united states .',
    'i live in the united states of america .'
]

for sentence in sentences:
    print('\n', sentence)
    for n in [2, 3, 4]:
        log10p = models[n].score(sentence)
        log2p = log10p / np.log10(2)
        print("n: %d\t logp: %.3f" % (n, log2p))


 i like my pet dog .
n: 2	 logp: -29.216
n: 3	 logp: -28.843
n: 4	 logp: -29.219

 i like my pet zebra .
n: 2	 logp: -32.303
n: 3	 logp: -31.157
n: 4	 logp: -32.101

 i like my pet lion .
n: 2	 logp: -38.352
n: 3	 logp: -41.017
n: 4	 logp: -42.320

 i live in the united states .
n: 2	 logp: -22.599
n: 3	 logp: -20.095
n: 4	 logp: -17.854

 i live in the united states of america .
n: 2	 logp: -40.958
n: 3	 logp: -26.627
n: 4	 logp: -24.311
